In [1]:
mnist_results_paths = ['../output/ablation-mnist-results.json',
                       '../output/ablation-mnist-results-spsa.json']
cifar10_results_paths = ['../output/ablation-cifar10-results.json',
                         '../output/ablation-cifar10-results2.json']

In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
import os
from scipy import stats

In [3]:
import json

def read_json(path):
    rows = []
    with open(path) as f:
        for line in f:
            if line.strip():
                row = json.loads(line[:-1])
                rows.append(row)
    return pd.DataFrame(rows)

def read_jsons(paths):
    dfs = [read_json(path) for path in paths]
    return pd.concat(dfs, sort=False).reset_index()

In [4]:
def process(df):
    df['attack_type'] = df.apply(
        lambda r: ('%s(eps=%.1f)' % (r['attack_type'], r['eps'])
                   if 'eps' in r else r['attack_type']), axis=1)
    counts = df.groupby(['model_path', 'attack_type']).agg({'accuracies': len})
    duplicates = counts[counts.accuracies >= 2]
    if len(duplicates) > 0:
        print(duplicates.reset_index().values)
    assert len(duplicates) == 0, 'Please remove duplicates'
    return df

# MNIST results

In [5]:
results = process(read_jsons(mnist_results_paths))

In [6]:
results.groupby('model_path').agg({'attack_type': len})

,attack_type
model_path,
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-max_fit_l1_1.pkl,5
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1-scrambling.pkl,5
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1.pkl,5
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-elliptical-maxout_4.pkl,5
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog-maxout_4.pkl,5
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relog.pkl,5
/nfs/home2/minhle/newlogic/output/ablation-mnist-models/relu.pkl,5
output/ablation-mnist-models/relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1-overlay.pkl,4
output/ablation-mnist-models/relog-minmaxout_2_4.pkl,4


In [7]:
def compute_accuracy_and_ttest(col):
    if col.name in ('index', 'model_name'):
        return list(col)
    formatted_accuracies = []
    for prev, curr in zip([None] + list(col), list(col)):
        if isinstance(curr, (str, float)):
            formatted_accuracies.append(curr)
        elif isinstance(curr, list):
            acc = np.mean(curr)
            acc = '%.2f' % acc
            if isinstance(prev, list):
                _, p_value = stats.ttest_ind(prev, curr, equal_var = False)
                if p_value < 0.05:
                    acc += '*'
            formatted_accuracies.append(acc)
        else:
            raise ValueError("Unsupported type: " + str(curr))
    return pd.Series(formatted_accuracies)

In [8]:
def format_table(df, formated_names, model_order, attack_order):
    df['model_name'] = df.model_path.apply(os.path.basename)
    # filtering
    df = df[df.model_name.isin(formated_names) &
            df.attack_type.isin(attack_order)].copy()
    # convert to wide format
    df = df.pivot(index='model_name', values='accuracies', columns='attack_type')
    # change column order
    df = (df.reindex(sorted(df.columns, key=attack_order.__getitem__), axis=1)
          .reset_index().fillna('-'))
    df['model_order'] = df.model_name.apply(model_order.__getitem__)
    df = df.sort_values(['model_order']).drop(columns='model_order')
    # compute mean accuracy and p-values
    df = df.apply(compute_accuracy_and_ttest)
    # format model names last
    df['model_name'] = df.model_name.apply(formatted_names.__getitem__)    
    return df, df.to_latex(index=False)
#     return df, None

## Ablation table

In [9]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog'), 
    ('relog-maxout_4.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout_2_4.pkl', '+ MinOut (k=2)'),
    ('relog-elliptical-maxout_4.pkl', '+ Elliptical'),
    ('relog-elliptical-maxout_4-max_fit_l1_1.pkl', '+ MaxFit'),  
    ('relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1.pkl', '+ MSE training'),  
    ('relog-elliptical-maxout_4-sigmoid_out-max_fit_l1_1-overlay.pkl', '+ Negative examples'),  
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [10]:
used_attacks = ['FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)', 'CW(eps=nan)',
                'BIM(eps=0.3)', 'SPSA(eps=0.3)']
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [11]:
ablation_table, latex = format_table(results, formatted_names, model_order, attack_order)

In [12]:
print(latex)

\begin{tabular}{llllll}
\toprule
          model\_name & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) & SPSA(eps=0.3) \\
\midrule
            Baseline &             0.33 &            0.82 &        0.05 &         0.01 &          0.36 \\
             + ReLog &            0.64* &           0.92* &       0.42* &        0.15* &         0.52* \\
      + MaxOut (k=4) &            0.56* &           0.89* &       0.10* &        0.07* &          0.52 \\
      + MinOut (k=2) &             0.56 &           0.90* &       0.09* &        0.12* &             - \\
        + Elliptical &            0.70* &           0.89* &       0.37* &        0.17* &          0.56 \\
            + MaxFit &            0.75* &            0.90 &       0.46* &        0.32* &         0.66* \\
      + MSE training &            0.59* &           0.87* &       0.90* &         0.33 &          0.65 \\
 + Negative examples &             0.60 &           0.84* &       0.87* &         0.34 &             - \\
\

# CIFAR-10 results

In [13]:
cifar10_results = process(read_jsons(cifar10_results_paths))

In [14]:
cifar10_results.model_path.unique()

array(['/nfs/home2/minhle/newlogic/output/ablation-cifar10-models/relog-maxout_4.pkl',
       '/nfs/home2/minhle/newlogic/output/ablation-cifar10-models/relog-minmaxout_2_4.pkl',
       '/nfs/home2/minhle/newlogic/output/ablation-cifar10-models2/relog-quadratic-maxout_4.pkl',
       '/nfs/home2/minhle/newlogic/output/ablation-cifar10-models2/relog-quadratic-maxout_4-sigmoid_out.pkl'],
      dtype=object)

In [15]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog'), 
    ('relog-maxout_4.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout_2_4.pkl', '+ MinOut (k=2)'),
    ('relog-quadratic-maxout_4.pkl', '+ Quadratic'),
    ('relog-quadratic-maxout_4-sigmoid_out.pkl', '+ MSE training')
])
model_order = {n: i for i, n in enumerate(formatted_names)}


In [16]:
ablation_table, latex = format_table(cifar10_results, formatted_names, model_order, attack_order)


In [17]:
print(latex)


\begin{tabular}{llllll}
\toprule
     model\_name & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) & SPSA(eps=0.3) \\
\midrule
 + MaxOut (k=4) &             0.10 &            0.29 &        0.02 &         0.00 &          0.57 \\
 + MinOut (k=2) &            0.09* &            0.30 &       0.01* &         0.00 &             - \\
    + Quadratic &            0.07* &           0.27* &        0.02 &        0.00* &          0.54 \\
 + MSE training &            0.16* &           0.22* &       0.03* &        0.00* &             - \\
\bottomrule
\end{tabular}

